# Run scDeepCluster on 10X PBMC dataset 

In [1]:
from time import time
import math, os
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from scDeepCluster import scDeepCluster,cluster_acc
import numpy as np
from sklearn import metrics
import h5py
import scanpy as sc
from preprocess import read_dataset, normalize

In [2]:
# for repeatability
torch.manual_seed(42)

Setup parameters.

In [3]:
'''
Parameter setting
'''

class Args(object):
    def __init__(self):
        self.n_clusters = 0
        self.knn = 20
        self.resolution = 0.8
        self.batch_size = 256
        self.data_file = './data/sample_151507.h5'
        self.maxiter = 2000
        self.pretrain_epochs = 300
        self.gamma = 1.
        self.sigma = 2.5
        self.update_interval = 1
        self.tol = 0.001
        self.ae_weights = None
        self.save_dir = 'results/scDeepCluster/'
        self.ae_weight_file = 'AE_weights.pth.tar'
        self.final_latent_file = 'final_latent_file.txt'
        self.predict_label_file = 'pred_labels.txt'
        self.device = 'cuda'

args = Args()

Normalizating and preprocessing count data.

In [4]:
data_mat = h5py.File(args.data_file)
x = np.array(data_mat['X'])
# y is the ground truth labels for evaluating clustering performance
if 'Y' in data_mat:
    y = np.array(data_mat['Y'])
else:
    y = None
data_mat.close()



# preprocessing scRNA-seq read counts matrix
adata = sc.AnnData(x)
if y is not None:
    adata.obs['Group'] = y

adata = read_dataset(adata)

adata = normalize(adata)

input_size = adata.n_vars

print(args)

print(adata.X.shape)
if y is not None:
    print(y.shape)

### Autoencoder: Successfully preprocessed 1159 genes and 4226 cells.
(4226, 1159)
(4226,)


D:\Temp\ipykernel_15896\3523718491.py:13: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = sc.AnnData(x)


Build scDeepCluster model

In [5]:
model = scDeepCluster(input_dim=adata.n_vars, z_dim=16,encodeLayer=[200], decodeLayer=[64,512], sigma=args.sigma, gamma=args.gamma, device=args.device)


print(str(model))

scDeepCluster(
  (encoder): Sequential(
    (0): Linear(in_features=1159, out_features=200, bias=True)
    (1): ReLU()
  )
  (decoder): Sequential(
    (0): Linear(in_features=16, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=512, bias=True)
    (3): ReLU()
  )
  (_enc_mu): Linear(in_features=200, out_features=16, bias=True)
  (_dec_mean): Sequential(
    (0): Linear(in_features=512, out_features=1159, bias=True)
    (1): MeanAct()
  )
  (_dec_disp): Sequential(
    (0): Linear(in_features=512, out_features=1159, bias=True)
    (1): DispAct()
  )
  (_dec_pi): Sequential(
    (0): Linear(in_features=512, out_features=1159, bias=True)
    (1): Sigmoid()
  )
  (zinb_loss): ZINBLoss()
)


Pretraining stage.

In [6]:
t0 = time()
model.pretrain_autoencoder(X=adata.X, X_raw=adata.raw.X, size_factor=adata.obs.size_factors, 
                        batch_size=args.batch_size, epochs=args.pretrain_epochs, ae_weights=args.ae_weight_file)

print('Pretraining time: %d seconds.' % int(time() - t0))


Pretraining stage
Pretrain epoch   1, ZINB loss: 1.25548597
Pretrain epoch   2, ZINB loss: 1.04477404
Pretrain epoch   3, ZINB loss: 1.01086700
Pretrain epoch   4, ZINB loss: 0.99763910
Pretrain epoch   5, ZINB loss: 0.99108203
Pretrain epoch   6, ZINB loss: 0.98763675
Pretrain epoch   7, ZINB loss: 0.98625194
Pretrain epoch   8, ZINB loss: 0.98465739
Pretrain epoch   9, ZINB loss: 0.98342778
Pretrain epoch  10, ZINB loss: 0.98231179
Pretrain epoch  11, ZINB loss: 0.98123679
Pretrain epoch  12, ZINB loss: 0.98045401
Pretrain epoch  13, ZINB loss: 0.97958028
Pretrain epoch  14, ZINB loss: 0.97921863
Pretrain epoch  15, ZINB loss: 0.97810017
Pretrain epoch  16, ZINB loss: 0.97762141
Pretrain epoch  17, ZINB loss: 0.97690947
Pretrain epoch  18, ZINB loss: 0.97603345
Pretrain epoch  19, ZINB loss: 0.97556395
Pretrain epoch  20, ZINB loss: 0.97525524
Pretrain epoch  21, ZINB loss: 0.97492006
Pretrain epoch  22, ZINB loss: 0.97420102
Pretrain epoch  23, ZINB loss: 0.97413122
Pretrain epoch  

Clustering stage.

In [7]:
if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)


### estimate number of clusters by Louvain algorithm on the autoencoder latent representations
pretrain_latent = model.encodeBatch(torch.tensor(adata.X)).cpu().numpy()
adata_latent = sc.AnnData(pretrain_latent)
sc.pp.neighbors(adata_latent, n_neighbors=args.knn, use_rep="X")
sc.tl.louvain(adata_latent, resolution=args.resolution)
y_pred_init = np.asarray(adata_latent.obs['louvain'],dtype=int)
features = pd.DataFrame(adata_latent.X,index=np.arange(0,adata_latent.n_obs))
Group = pd.Series(y_pred_init,index=np.arange(0,adata_latent.n_obs),name="Group")
Mergefeature = pd.concat([features,Group],axis=1)
cluster_centers = np.asarray(Mergefeature.groupby("Group").mean())
n_clusters = cluster_centers.shape[0]
print('Estimated number of clusters: ', n_clusters)
y_pred, _, _, _, _ = model.fit(X=adata.X, X_raw=adata.raw.X, size_factor=adata.obs.size_factors, n_clusters=n_clusters, init_centroid=cluster_centers, 
            y_pred_init=y_pred_init, y=y, batch_size=args.batch_size, num_epochs=args.maxiter, update_interval=args.update_interval, tol=args.tol, save_dir=args.save_dir)


print('Total time: %d seconds.' % int(time() - t0))

Estimated number of clusters:  9
Clustering stage
Initializing cluster centers with kmeans.
Initializing k-means: NMI= 0.5123, ARI= 0.3989
Clustering   : NMI= 0.4878, ARI= 0.3624
Total time: 65 seconds.


Output and save predicted labels and latent features.

In [8]:
if y is not None:
    nmi, ari=cluster_acc(y, y_pred)

final_latent = model.encodeBatch(torch.tensor(adata.X)).cpu().numpy()
np.savetxt(args.final_latent_file, final_latent, delimiter=",")
np.savetxt(args.predict_label_file, y_pred, delimiter=",", fmt="%i")

Run t-SNE on latent features.

In [9]:
from openTSNE import TSNE

tsne_embedding = TSNE(
                    n_components=2,
                    perplexity=30,
                    initialization="pca",
                    metric="euclidean",
                    n_jobs=8,
                    random_state=42,
                )
latent_tsne_2 = tsne_embedding.fit(final_latent)
np.savetxt("tsne_2D.txt", latent_tsne_2, delimiter=",")

Plot 2D t-SNE of latent features

In [ ]:
"""
rm(list = ls())
library(ggplot2)

latent_tsne <- read.table("tsne_2D.txt", sep = ",")
colnames(latent_tsne) <- c("TSNE_1", "TSNE_2")
y_pred <- as.numeric(readLines("pred_labels.txt"))
y_pred <- factor(y_pred, levels = 0:max(y_pred))

dat <- data.frame(latent_tsne, y_pred = y_pred)

m <- ggplot(dat, aes(x = TSNE_1, y = TSNE_2, color = y_pred)) +
    geom_point() +
    theme_classic()
print(m)
"""